In [1]:
# bonsai batcher
# Ask Holm Carlsen

In [2]:
import numpy as np
from osgeo import gdal
import os, time, os.path
from sklearn.ensemble import RandomForestClassifier




In [3]:
os.system("conda list")

0

In [4]:
# Dir til de færdige billeder. Der kommer undermapper med bloknavne. 
bdir = r"E:\Bonsai2022FraAsk\OUTPUTS\batcherDir\\"

# NAME AND PATH FOR TRAINING
training = np.loadtxt(r'E:\Bonsai2022FraAsk\train\all_train_F22_grus_ubefaest.txt')
# Splitting the data into labels and raster data.
ytrain = training[:, 0]
xtrain = training[:, 1:]


print(training.shape)


(9641, 5)


In [5]:
unique_elements, counts_elements = np.unique(ytrain, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray(((unique_elements), counts_elements)))

Frequency of unique values of the said array:
[[1.000e+00 2.000e+00]
 [3.171e+03 6.470e+03]]


In [6]:
# forskellige funktioner

def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte, [ 'COMPRESS=DEFLATE' ])
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

def predictor(dirt,raster, batch_dir):
    rast_in = dirt+raster
    rast_out = batch_dir+raster
    print(raster)
    # Her downsamples til 40cm. Nok bedre med bedre opløsning 
    command = "gdal_translate -outsize 2500 2500 "+rast_in+" "+batch_dir+raster     
    os.system(command)
    
    # Åbner raster og tager geo_transform og proj
    raster_dataset = gdal.Open(rast_out, gdal.GA_ReadOnly)
    geo_transform = raster_dataset.GetGeoTransform()
    proj = raster_dataset.GetProjectionRef()
    
    # Ændrer billedet til "flade arrays". RFC bruger ikke info om hvor i billedet en pixel er fra.
    # rows, cols bruges senere til at ændre billedet tilbage til dets normale form/dimension.
    bands_data = []
    for b in range(1, raster_dataset.RasterCount + 1):
        band = raster_dataset.GetRasterBand(b)
        bands_data.append(band.ReadAsArray())
    bands_data = np.dstack(bands_data)
    rows, cols, n_bands = bands_data.shape
    n_samples = rows * cols
    flat_pixels = bands_data.reshape((n_samples, n_bands))
    fname = batch_dir + raster

    RFCtimeSTART = time.time()
    # Her defineres modellen med 200 træer og alle cpu processorer (n_jobs=-1)
    classifierRFC = RandomForestClassifier(n_estimators=200, n_jobs=-1) 
    # her trænes/fittes den
    classifierRFC.fit(xtrain, ytrain)
    # her klassificerer den de "flade pixels"
    resultRFC = classifierRFC.predict(flat_pixels)
    # her rykkes de flade pixels tilbage til billedets originale form
    classificationRFC = resultRFC.reshape((rows, cols))
    RFCtimeEND = time.time() - RFCtimeSTART
    
    raster_dataset = None
    
    # Her gemme billedet.
    write_geotiff(fname, classificationRFC, geo_transform, proj)
    print(raster,"RFC computation time:", RFCtimeEND / 60, "minutes")






In [7]:
# liste over blokke på NAS'en

#inlist = [r'Q:\2022\2022_foraars_orto\81_01\tifjpeg\\']
#inlist = [r'E:\Bonsai2022FraAsk\testImages\images\\'] # test
#inlist = ['P:\ASS\\2019\\2019_foraars_orto\\82_15\\tifjpeg\\']
inlist = [ 'Q:\\2022\\2022_foraars_orto\\\\82_14\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_15\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_16\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_17\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_18\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_19\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_20\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_21\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_22\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_23\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_24\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_01\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_02\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_03\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_04\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_05\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_06\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_07\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_08\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_09\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_10\\tifjpeg\\']
#inlist = [r'Q:\2022\2022_foraars_orto\\']
#for i in os.listdir(inlist[0]):
#    if len(i) == 5:
#        inlist.append(inlist[0]+i+'\\tifjpeg\\')
#inlist = inlist[2:]

In [8]:
inlist

['Q:\\2022\\2022_foraars_orto\\\\82_14\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_15\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_16\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_17\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_18\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_19\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_20\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_21\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_22\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_23\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\82_24\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_01\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_02\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_03\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_04\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_05\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_06\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_07\\tifjpeg\\',
 'Q:\\2022\\2022_foraars_orto\\\\81_08\\tifjpeg\\',
 'Q:\\2022\\

In [10]:
for rast_list in inlist:
    print(rast_list)
    bname = rast_list[-14:][:5]
    batch_dir = bdir+bname+"\\"
    #print(batch_dir)
    if not os.path.exists(batch_dir):
        os.makedirs(batch_dir)
    for i in os.listdir(rast_list):
        if i.endswith(".tif"):
            img = rast_list + i
            print(img)
            #print(rast_list,i,batch_dir)
            predictor(rast_list, i,batch_dir)

Q:\2022\2022_foraars_orto\\82_14\tifjpeg\
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6282_532.tif
2022_1km_6282_532.tif
2022_1km_6282_532.tif RFC computation time: 0.15780168771743774 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6282_531.tif
2022_1km_6282_531.tif
2022_1km_6282_531.tif RFC computation time: 0.15686509609222413 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6282_530.tif
2022_1km_6282_530.tif
2022_1km_6282_530.tif RFC computation time: 0.16081156333287558 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6281_532.tif
2022_1km_6281_532.tif
2022_1km_6281_532.tif RFC computation time: 0.16302593549092612 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6281_531.tif
2022_1km_6281_531.tif
2022_1km_6281_531.tif RFC computation time: 0.15309834082921345 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6281_530.tif
2022_1km_6281_530.tif
2022_1km_6281_530.tif RFC computation time: 0.15854359070460003 minutes
Q:\2022\2022_for

2022_1km_6278_511.tif RFC computation time: 0.130563755830129 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_535.tif
2022_1km_6277_535.tif
2022_1km_6277_535.tif RFC computation time: 0.1584678014119466 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_534.tif
2022_1km_6277_534.tif
2022_1km_6277_534.tif RFC computation time: 0.1627988298734029 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_533.tif
2022_1km_6277_533.tif
2022_1km_6277_533.tif RFC computation time: 0.1670745849609375 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_532.tif
2022_1km_6277_532.tif
2022_1km_6277_532.tif RFC computation time: 0.16380637089411418 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_531.tif
2022_1km_6277_531.tif
2022_1km_6277_531.tif RFC computation time: 0.16266645193099977 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6277_530.tif
2022_1km_6277_530.tif
2022_1km_6277_530.tif RFC computation time: 0.1597784201304118 

2022_1km_6276_511.tif RFC computation time: 0.13134416341781616 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_538.tif
2022_1km_6275_538.tif
2022_1km_6275_538.tif RFC computation time: 0.1522092302640279 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_537.tif
2022_1km_6275_537.tif
2022_1km_6275_537.tif RFC computation time: 0.1528640667597453 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_536.tif
2022_1km_6275_536.tif
2022_1km_6275_536.tif RFC computation time: 0.15098170439402261 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_535.tif
2022_1km_6275_535.tif
2022_1km_6275_535.tif RFC computation time: 0.15339978535970053 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_534.tif
2022_1km_6275_534.tif
2022_1km_6275_534.tif RFC computation time: 0.15668510993321735 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6275_533.tif
2022_1km_6275_533.tif
2022_1km_6275_533.tif RFC computation time: 0.16108650763829

2022_1km_6274_514.tif RFC computation time: 0.1610020120938619 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6274_513.tif
2022_1km_6274_513.tif
2022_1km_6274_513.tif RFC computation time: 0.16461082299550375 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6274_512.tif
2022_1km_6274_512.tif
2022_1km_6274_512.tif RFC computation time: 0.16140024264653524 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6274_511.tif
2022_1km_6274_511.tif
2022_1km_6274_511.tif RFC computation time: 0.15902072588602703 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6273_538.tif
2022_1km_6273_538.tif
2022_1km_6273_538.tif RFC computation time: 0.15768946011861165 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6273_537.tif
2022_1km_6273_537.tif
2022_1km_6273_537.tif RFC computation time: 0.15513500372568767 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6273_536.tif
2022_1km_6273_536.tif
2022_1km_6273_536.tif RFC computation time: 0.1534424463907

2022_1km_6272_517.tif RFC computation time: 0.140695854028066 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_516.tif
2022_1km_6272_516.tif
2022_1km_6272_516.tif RFC computation time: 0.15737966299057007 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_515.tif
2022_1km_6272_515.tif
2022_1km_6272_515.tif RFC computation time: 0.16023656924565632 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_514.tif
2022_1km_6272_514.tif
2022_1km_6272_514.tif RFC computation time: 0.1606842041015625 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_513.tif
2022_1km_6272_513.tif
2022_1km_6272_513.tif RFC computation time: 0.16121323506037394 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_512.tif
2022_1km_6272_512.tif
2022_1km_6272_512.tif RFC computation time: 0.16192697683970134 minutes
Q:\2022\2022_foraars_orto\\82_14\tifjpeg\2022_1km_6272_511.tif
2022_1km_6272_511.tif
2022_1km_6272_511.tif RFC computation time: 0.161439824104309

...

2022_1km_6350_624.tif RFC computation time: 0.17218258380889892 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_623.tif
2022_1km_6350_623.tif
2022_1km_6350_623.tif RFC computation time: 0.17038179238637288 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_622.tif
2022_1km_6350_622.tif
2022_1km_6350_622.tif RFC computation time: 0.1719123601913452 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_621.tif
2022_1km_6350_621.tif
2022_1km_6350_621.tif RFC computation time: 0.1729374647140503 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_620.tif
2022_1km_6350_620.tif
2022_1km_6350_620.tif RFC computation time: 0.17250771125157674 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_619.tif
2022_1km_6350_619.tif
2022_1km_6350_619.tif RFC computation time: 0.17394913832346598 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6350_618.tif
2022_1km_6350_618.tif
2022_1km_6350_618.tif RFC computation time: 0.17260047594706

2022_1km_6348_621.tif RFC computation time: 0.16795833508173624 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_620.tif
2022_1km_6348_620.tif
2022_1km_6348_620.tif RFC computation time: 0.17363280057907104 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_619.tif
2022_1km_6348_619.tif
2022_1km_6348_619.tif RFC computation time: 0.17230506738026938 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_618.tif
2022_1km_6348_618.tif
2022_1km_6348_618.tif RFC computation time: 0.16921333074569703 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_617.tif
2022_1km_6348_617.tif
2022_1km_6348_617.tif RFC computation time: 0.1677038113276164 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_616.tif
2022_1km_6348_616.tif
2022_1km_6348_616.tif RFC computation time: 0.16948265234629314 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6348_615.tif
2022_1km_6348_615.tif
2022_1km_6348_615.tif RFC computation time: 0.1691800395647

2022_1km_6346_615.tif RFC computation time: 0.16005706787109375 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_614.tif
2022_1km_6346_614.tif
2022_1km_6346_614.tif RFC computation time: 0.1576164444287618 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_613.tif
2022_1km_6346_613.tif
2022_1km_6346_613.tif RFC computation time: 0.14188913504282633 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_612.tif
2022_1km_6346_612.tif
2022_1km_6346_612.tif RFC computation time: 0.13140533765157064 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_593.tif
2022_1km_6346_593.tif
2022_1km_6346_593.tif RFC computation time: 0.15755849679311115 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_592.tif
2022_1km_6346_592.tif
2022_1km_6346_592.tif RFC computation time: 0.1675718108812968 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6346_591.tif
2022_1km_6346_591.tif
2022_1km_6346_591.tif RFC computation time: 0.16602312326431

2022_1km_6343_621.tif RFC computation time: 0.14778413772583007 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_620.tif
2022_1km_6343_620.tif
2022_1km_6343_620.tif RFC computation time: 0.15354581276575724 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_619.tif
2022_1km_6343_619.tif
2022_1km_6343_619.tif RFC computation time: 0.1596987247467041 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_618.tif
2022_1km_6343_618.tif
2022_1km_6343_618.tif RFC computation time: 0.1454986810684204 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_617.tif
2022_1km_6343_617.tif
2022_1km_6343_617.tif RFC computation time: 0.14229657649993896 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_616.tif
2022_1km_6343_616.tif
2022_1km_6343_616.tif RFC computation time: 0.13576073249181111 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6343_615.tif
2022_1km_6343_615.tif
2022_1km_6343_615.tif RFC computation time: 0.14010441303253

2022_1km_6339_589.tif RFC computation time: 0.13895989259084066 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6339_588.tif
2022_1km_6339_588.tif
2022_1km_6339_588.tif RFC computation time: 0.16041518847147623 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6339_587.tif
2022_1km_6339_587.tif
2022_1km_6339_587.tif RFC computation time: 0.16008164485295615 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6339_586.tif
2022_1km_6339_586.tif
2022_1km_6339_586.tif RFC computation time: 0.16353970766067505 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6338_617.tif
2022_1km_6338_617.tif
2022_1km_6338_617.tif RFC computation time: 0.13550435702006022 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6338_616.tif
2022_1km_6338_616.tif
2022_1km_6338_616.tif RFC computation time: 0.1353800892829895 minutes
Q:\2022\2022_foraars_orto\\81_10\tifjpeg\2022_1km_6338_615.tif
2022_1km_6338_615.tif
2022_1km_6338_615.tif RFC computation time: 0.1325875560442

In [ ]:
for rast_list in inlist:
    print(rast_list)
    bname = rast_list[-15:][:5]
    batch_dir = bdir+bname+"\\"
    if not os.path.exists(batch_dir):
        os.makedirs(batch_dir)
    for i in os.listdir(rast_list):
        if i.endswith(".tif"):
            img = rast_list + i
            #print(img)
            print(rast_list,i,batch_dir)
            #predictor(rast_list, i,batch_dir)